In [7]:
GYRO = False
SEQUENCE_LENGTH = 4
SEQUENCE_OVERLAP = 3
BATCH_SIZE = 10
EPOCHS = 30
CATEGORY = "moving" # "moving" or "stationary"
MODEL_NAME = f"category:{CATEGORY}_epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}"


### This code takes in the type of activity, ie "stationary" or "moving" and trains a model for just classifying which phsyical activity of that category the data is  

In [8]:
import file_tagger
import sequence_genrator
import tensorflow as tf
import numpy as np
from keras import layers, Sequential, models
from sklearn.model_selection import train_test_split


STATIONARY_ACTIVITIES = {
    "sitting",
    "standing",
    "lying_down_back",
    "lying_down_stomach",
    "lying_down_right",
    "lying_down_left"
}

STATIONARY_ACTIVITIES_COMBINED = {
    "sitting_or_standing",
    "lying_down_back",
    "lying_down_stomach",
    "lying_down_right",
    "lying_down_left"
}

MOVING_ACTIVITIES = {
    "walking",
    "ascending_stairs",
    "descending_stairs",
    "shuffle_walking",
    "running",
    "misc_movements"
}


DATA_DIRECTORY = "./all_respeck"

activity_dict = {"moving": MOVING_ACTIVITIES,
                       "stationary": STATIONARY_ACTIVITIES
            }

POSSIBLE_ACTIVITIES = activity_dict[CATEGORY]


In [9]:
# split data into training, dev, and test sets
def train_dev_test_split(data, labels, dev_size, test_size):
    # Split the data into training and temporary (dev + test) sets
    train_data, temp_data, train_labels, temp_labels = train_test_split(data, labels, test_size=(dev_size + test_size))
    
    # Split the temporary data into dev and test sets
    dev_data, test_data, dev_labels, test_labels = train_test_split(temp_data, temp_labels, 
                                                                 test_size=(test_size / (dev_size + test_size)))
    
    return train_data, dev_data, test_data, train_labels, dev_labels, test_labels

In [10]:
def generate_training_data(directory, sequence_length, overlap, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in csv_dictionary.keys():
        
        activity = key.split("&")[0]
        if activity in POSSIBLE_ACTIVITIES:

            # iterates through each csv file for the activity 
            for csv_file in csv_dictionary[key]:
                if gyro:
                    sequences = sequence_genrator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap)
                else:
                    sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap)

                # iterate through each generated sequence
                for sequence in sequences:
                    if activity == "standing" or activity == "sitting":
                        tagged_data.append(("sitting_or_standing", sequence))
                    else:
                        tagged_data.append((activity, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")
    return tagged_data

In [11]:
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    if GYRO:
        width = 6
    else:
        width = 3
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [12]:
tagged_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP)

# Combine all sequences and labels
sequences = [sequence for _, sequence in tagged_sequences]
labels = [label for label, _ in tagged_sequences]

if CATEGORY == "stationary":
    possible_activities = STATIONARY_ACTIVITIES_COMBINED
else:
    possible_activities = POSSIBLE_ACTIVITIES

# encode labels to numbers
sequences = np.array(sequences, dtype=np.float32)
label_to_index = {label: idx for idx, label in enumerate(possible_activities)}
labels_encoded = [label_to_index[label] for label in labels]
labels_encoded = np.array(labels_encoded)

train_data, dev_data, test_data, train_labels, dev_labels, test_labels = train_dev_test_split(sequences, labels_encoded, dev_size=0.1, test_size=0.1) #10% dev, 10% test


# train and save model (CHOOSE BETWEEN CNN AND LSTM)
model = train_model_CNN(train_data, train_labels, possible_activities, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) #batch_size, epochs


# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the trained model
model.save(f"models/models_for_presentation/moving_classifier.keras")

there are 7439 tagged sequences in the dataset
Epoch 1/30
596/596 [==============================] - 2s 3ms/step - loss: 1.0065 - accuracy: 0.5991 - val_loss: 0.5955 - val_accuracy: 0.7863
Epoch 2/30
596/596 [==============================] - 2s 3ms/step - loss: 0.4384 - accuracy: 0.8392 - val_loss: 0.2966 - val_accuracy: 0.8777
Epoch 3/30
596/596 [==============================] - 2s 3ms/step - loss: 0.2648 - accuracy: 0.9084 - val_loss: 0.2162 - val_accuracy: 0.9247
Epoch 4/30
596/596 [==============================] - 2s 3ms/step - loss: 0.1988 - accuracy: 0.9298 - val_loss: 0.2074 - val_accuracy: 0.9288
Epoch 5/30
596/596 [==============================] - 2s 3ms/step - loss: 0.1533 - accuracy: 0.9474 - val_loss: 0.2064 - val_accuracy: 0.9234
Epoch 6/30
596/596 [==============================] - 2s 3ms/step - loss: 0.1228 - accuracy: 0.9580 - val_loss: 0.1448 - val_accuracy: 0.9449
Epoch 7/30
596/596 [==============================] - 2s 3ms/step - loss: 0.1207 - accuracy: 0.9583 -